In [1]:
%pylab inline

import numpy as np
import pandas as pd
import os
from joblib import Parallel, delayed
import tqdm


Populating the interactive namespace from numpy and matplotlib


# NUMBER OF LEVELS AND SIZE OF CONTAINERS

In [2]:
###PREPROCESSING
SCALES_FOLDER = '/data/work/user/laura_data/scales_by_user_0.5/'
scales = pd.read_parquet(SCALES_FOLDER)
scales['sizes'] = scales['sizes'].apply(lambda x:sorted(x))
scales['n_scales'] = scales['scales'].apply(lambda x:len(x)+1)
scales['size2_level'] = scales['sizes'].apply(lambda x:x[0] if len(x)>0 else np.nan)
scales['size3_level'] = scales['sizes'].apply(lambda x:x[1] if len(x)>1 else np.nan)
scales['size4_level'] = scales['sizes'].apply(lambda x:x[2] if len(x)>2 else np.nan)
scales['size5_level'] = scales['sizes'].apply(lambda x:x[3] if len(x)>3 else np.nan)

In [3]:
all_sizes = pd.concat([scales['size{}_level'.format(i)] for i in range(2,6)])
all_levels = pd.concat([pd.Series([i]*len(scales)) for i in range(2,6)])
sizes = pd.concat([all_levels,all_sizes],axis = 1).rename(columns = {0:'level',1:'size'})

In [5]:
sizes.dropna().to_pickle("../outputs/Figure1/data_size.pkl")
sizes.dropna().to_csv("../outputs/Figure1/data_size.csv",index=False)

In [6]:
scales['n_scales'].to_pickle("../outputs/Figure1/data_levels.pkl")
scales['n_scales'].to_csv("../outputs/Figure1/data_levels.csv",index=False)


# INTEREVENT_TIMES_BETWEEN_CONTAINERS

In [7]:
TRIPS_FILE = '/data/work/user/laura_data/scale_trips_0.5/'

def read_bucket_of_data(bucket):
    print(bucket)
    trips = pd.read_parquet(bucket)
    trips = trips.sort_values(by=['useruuid','start']).copy()
    inter_event_time = trips[trips.trip_scale<=6].groupby(['useruuid','trip_scale']).apply(compute_quantity_by_user)
    return inter_event_time

def compute_quantity_by_user(group):
    diff = np.nanmedian(group['start'] - group['end'].shift())
    return diff


In [8]:
inter_event_time_all = read_bucket_of_data(TRIPS_FILE).reset_index(name='inter_time')
inter_event_time_all[['trip_scale','inter_time']].dropna().to_pickle('../outputs/Figure1/data_time.pkl')
inter_event_time_all[['trip_scale','inter_time']].dropna().to_csv('../outputs/Figure1/data_time.csv',index=False)

/data/work/user/laura_data/scale_trips_0.5/


MemoryError: Unable to allocate 8.04 GiB for an array with shape (3, 359895028) and data type object